***
## ASFSession

Because downloading any product in the ASF archive requires authentication, `asf_search` provides the `ASFSession` class, a subclass of `Session` with a few specific methods added to make authentication straightforward.

Using .netrc credentials is the preferred method for authentication. For more information, see the [session authentication documentation](https://docs.asf.alaska.edu/asf_search/downloading/#session-authentication)

A new, unauthenticated session can be created, although the authentication methods listed below allow chaining for direct creation of an authenticated session.

In [1]:
username = 'username' # string
password = 'password' # string
rslc_1 = 'https://datapool.asf.alaska.edu/L1.0/A3/ALPSRP274410710-L1.0.zip' # string
rslc_2 = 'https://datapool.asf.alaska.edu/L1.0/A3/ALPSRP267700710-L1.0.zip' # string
key = '' # string
secret = '' # string
token = '' # string
region = '' # string

# hysds specifications
_time_limit = 3600
_soft_time_limit = 3600
_disk_usage = "10GB"
_submission_type = "iteration"
_label = "ALOS to RSLC PGE"

### `auth_with_creds()`
This authentication method accepts a username and password and establishes an authentication session with EDL and ASF.

In [2]:
import os
import yaml
import asf_search as asf
import boto3
import aws_uploader

WORKING_DIR = os.getcwd()
HOME_DIR = os.environ['HOME']
NOTEBOOK_PGE_DIR = os.environ.get('NOTEBOOK_PGE_DIR', WORKING_DIR)
ISCE3_BUILD_DIR = os.environ.get('ISCE3_BUILD_DIR', f'{HOME_DIR}/isce3/build/install')

DOWNLOAD_DIR = os.path.join(WORKING_DIR, 'downloads')
EXTRACT_DIR = os.path.join(WORKING_DIR, 'alos_data')
OUTPUT_DIR = os.path.join(WORKING_DIR, 'output')

os.makedirs(DOWNLOAD_DIR, exist_ok=True)
os.makedirs(EXTRACT_DIR, exist_ok=True)
os.makedirs(OUTPUT_DIR, exist_ok=True)

with open(os.path.join(NOTEBOOK_PGE_DIR, '../templates/focus.yaml'), 'r') as f:
    FOCUS_YML = yaml.safe_load(f)

with open(os.path.join(NOTEBOOK_PGE_DIR, '../templates/insar.yaml'), 'r') as f:
    INSAR_YML = yaml.safe_load(f)

def download_alos_data(urls: list):
    """Downloads ALOS-1 data given by two urls and returns the paths to the ."""
    try:
        current_dir = os.getcwd()
        user_pass_session = asf.ASFSession().auth_with_creds(username, password)
        asf.download_urls(urls=urls, path=DOWNLOAD_DIR, session=user_pass_session, processes=2)
        files = os.listdir(DOWNLOAD_DIR)
        os.chdir(EXTRACT_DIR)
        extract_dirs = []
        for f in files:
            splitext = os.path.splitext(f)
            zip_f = os.path.join(DOWNLOAD_DIR, f)
            !unzip -o {zip_f}
            extract_dirs.append(os.path.abspath(splitext[0]))
        os.chdir(current_dir)
        print('Extracted:', extract_dirs)
        return extract_dirs
    except asf.ASFAuthenticationError as e:
        print(f'Auth failed: {e}')

def write_focus_config(target_path: str, yml_path: str):
    """Downloads"""
    FOCUS_YML['runconfig']['groups']['input_file_group']['input_file_path'] = [target_path]
    FOCUS_YML['runconfig']['groups']['product_path_group']['sas_output_file'] = os.path.join('RSLC', os.path.basename(target_path))
    FOCUS_YML['runconfig']['groups']['product_path_group']['sas_config_file'] = yml_path
    with open(yml_path, 'w', encoding='utf-8') as f:
        yaml.dump(FOCUS_YML, f, default_flow_style=False)

def write_insar_config(target_path: str, yml_path: str):
    INSAR_YML['runconfig']['groups']['input_file_group']['reference_rslc_file'] = f1
    INSAR_YML['runconfig']['groups']['input_file_group']['secondary_rslc_file'] = f2
    with open(yml_path, 'w', encoding='utf-8') as f:
        yaml.dump(INSAR_YML, f, default_flow_style=False)

print(WORKING_DIR)

/home/jovyan/Github/alos-to-insar/notebook_pges


In [3]:
# os.environ['PYTHONPATH'] = f"{ISCE3_BUILD_DIR}/install/packages:${os.environ['LD_LIBRARY_PATH']}"
# os.environ['LD_LIBRARY_PATH']= f"{ISCE3_BUILD_DIR}/install/lib64:${os.environ['LD_LIBRARY_PATH']}"

In [4]:
# Download the specified ALOS data
extract_dirs = download_alos_data([rslc_1, rslc_2])
print(extract_dirs)

# Focus the data
rlsc_paths = []
for i, d in enumerate(extract_dirs):
    target_path = os.path.join(OUTPUT_DIR, f'{i}.h5')
    rlsc_paths.append(target_path)
    yml_path = os.path.join(OUTPUT_DIR, f'{i}.yaml')
    write_focus_config(target_path, yml_path)
    !mamba run -n isce3_src python {ISCE3_BUILD_DIR}/share/nisar/examples/alos_to_nisar_l0b.py -i {d} -o {target_path}
    !mamba run -n isce3_src python {ISCE3_BUILD_DIR}/packages/nisar/workflows/focus.py {yml_path} --no-log

/home/jovyan/.local/envs/isce3_src/lib/python3.9/site-packages/asf_search/download/download.py:65: UserWarning: File already exists, skipping download: /home/jovyan/Github/alos-to-insar/notebook_pges/downloads/ALPSRP274410710-L1.0.zip
  warnings.warn(f'File already exists, skipping download: {os.path.join(path, filename)}')
/home/jovyan/.local/envs/isce3_src/lib/python3.9/site-packages/asf_search/download/download.py:65: UserWarning: File already exists, skipping download: /home/jovyan/Github/alos-to-insar/notebook_pges/downloads/ALPSRP267700710-L1.0.zip
  warnings.warn(f'File already exists, skipping download: {os.path.join(path, filename)}')


Archive:  /home/jovyan/Github/alos-to-insar/notebook_pges/downloads/ALPSRP267700710-L1.0.zip
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of /home/jovyan/Github/alos-to-insar/notebook_pges/downloads/ALPSRP267700710-L1.0.zip or
        /home/jovyan/Github/alos-to-insar/notebook_pges/downloads/ALPSRP267700710-L1.0.zip.zip, and cannot find /home/jovyan/Github/alos-to-insar/notebook_pges/downloads/ALPSRP267700710-L1.0.zip.ZIP, period.
Archive:  /home/jovyan/Github/alos-to-insar/notebook_pges/downloads/ALPSRP274410710-L1.0.zip
   creating: ALPSRP274410710-L1.0/
  inflating: ALPSRP274410710-L1.0/IMG-HH-ALPSRP274410710-H1.0__A  
  inflating: ALPSRP274410710-L1.0/VOL-ALPSRP274410710-H1.0__A  
  inflating: ALPSRP274410710-L1.0/LED-ALPSRP2744107

In [5]:
from urllib.parse import urlparse

s3_url = 's3://nisar-st-data-ondemand/ADT_Testing/'
parsed_url = urlparse(s3_url)
bucket = parsed_url.netloc
path = parsed_url.path

# Upload to s3 bucket
if key != '' or secret != '' or token != '' or region != '':
    # short term access key
    uploader = aws_uploader.AWS(key, secret, token, region)
else:
    # uses role based access
    uploader = aws_uploader.AWS('', '', '', '', configdir=False)
uploader.upload_dir(OUTPUT_DIR, bucket, path)

S3UploadFailedError: Failed to upload /home/jovyan/Github/alos-to-insar/notebook_pges/output/0.yaml to nisar-st-data-ondemand/ADT_Testing/0.yaml: An error occurred (AccessDenied) when calling the PutObject operation: Access Denied